In [27]:
import numpy as np
import os

import pygimli as pg
from pygimli.meshtools import appendTriangleBoundary, merge2Meshes
from pygimli.mplviewer import drawMesh
from pygimli.viewer import showMesh
from pygimli.meshtools import convertMesh

ImportError: cannot import name 'convertMesh'

In [13]:
#Ici on commence par définir un maillage rectangulaire homogène

xmin, xmax = -0.02, 0.0 #On définit les limites en x du maillage
zmin, zmax = -0.3, 0. #On définit les limites en z du maillage

dx = 0.01 #On définit le pas de la maille
xreg = np.arange(xmin, xmax + dx, dx, 'float') #On calcule les coordonées des mailles
zreg = np.arange(zmin, zmax + dx, dx, 'float')

mesh1 = pg.Mesh(2) #On crée un maillage vide 
mesh1.create2DGrid(xreg, zreg, 0) #On crée le maillage
for c in mesh1.cells(): #On marque les cellules
    c.setMarker(1)

print(mesh1) #On affiche le mombre de noeuds, cellules et lignes

#showMesh(mesh1, mesh1.cellMarkers(), logScale=False, label="Region marker") #On définit les paramètres d'affichage

showMesh(mesh1, mesh1.cellMarkers(), label="Region marker")
#drawMesh(ax,mesh1)

Mesh: Nodes: 93 Cells: 60 Boundaries: 152
No valid data:  1 1 False


(<matplotlib.axes._subplots.AxesSubplot at 0x7f98ae444550>, None)

In [14]:
#A présent on crée une zone grâce à un polygone

poly = pg.Mesh(2)  # empty 2d mesh
nStart = poly.createNode(0.0, 0.0, 0.0) #On crée un noeud de départ, on travaille en 2D donc le dernier terme vaut 0.0

nA = nStart #On définit le noeud de départ
for z in zreg[1:]: #On démarre de 1 et on se balade sur l'axe des x en créant un noeud à chaque fois
    nB = poly.createNode(xmax, z, 0.0)
    poly.createEdge(nA, nB) #On définit un côté entre le noeud précédemment crée et le nouveau
    nA = nB #On remplace le noeud de départ par le noeud nouvellement crée

z2 = -0.3 #On définit une altitude z2 
nA = poly.createNode(xmax, z2, 0.0) #On crée un noeud
poly.createEdge(nB, nA) #On fait le lien entre le dernier noeud crée et celui-là
nB = poly.createNode(xmin, z2, 0.0) #On crée un autre noeud en symétrique
poly.createEdge(nA, nB) #On fait le lien avec le noeud précédent
nC = poly.createNode(xmin, -0.8, 0.0)
poly.createEdge(nB, nC)
nD = poly.createNode(0.4, -0.8, 0.0)
poly.createEdge(nC, nD)
nE = poly.createNode(0.4, 0.0, 0.0)
poly.createEdge(nD, nE)
poly.createEdge(nE, nStart) #On ferme le polygone!

tri = pg.TriangleWrapper(poly) #On appelle la fonction triangle
tri.setSwitches('-pzeAfaq31') #On rentre tout un tas de commandes rigolotes pour générer les triangles
# Ici on a :
# p : planar straight line graph ==> fichier poly
# z : on démarre le comptage à 0
# A : assigne un attribut à chaque triangle qui indique à quel segment il appartient et est lié
# f : algorithme de triangulation (?)
# a : impose une surface contrainte pour chaque triangle on peut ajouter un nombre si on veut préciser
# q31 : impose que les triangles générés aient au minimun des angles de 20° On peut ajouter un nombre derrière pour préciser le nombre que l'o souhaite

In [15]:
#A présent on génère le maillage hétérogène

mesh2 = pg.Mesh(2) #On appelle le second maillage autour du premier
tri.generate(mesh2) #On génère les triangles au sein du polygone précédemment crée

for cell in mesh2.cells(): #On génère les cellules de chaque maille
    cell.setMarker(2)

In [18]:
#Fusion des deux maillages pour créer un maillage hybride
#mesh3 = merge2Meshes(mesh1, mesh2) #On peut faire fusionner les deux maillages ainsi crées en un seul

a=showMesh(mesh2, mesh1.cellMarkers(), label="Region marker")

No valid data:  1 1 False


In [17]:
#Création d'une fonction pour interpoler la charge au fond du trou

#Définition des paramètres initiaux
p=1 #coordonnées xy des noeuds du maillage
d=30 #profondeur du trou en cm
radius=2 #rayon du trou en cm
h_0=-95 #charge initiale en cm, soit l'état initial du sol (teneur en eau exprimée en charge)
h_1=10 #hauteur d'eau au fond du trou en cm

In [24]:
print(mesh2[1])

TypeError: 'Mesh' object does not support indexing

In [26]:
mesh2(2)

TypeError: 'Mesh' object is not callable